## Collaborative Filtering

In [1]:
# import pandas
import pandas as pd
from sklearn.neighbors import NearestNeighbors

We will create an recommender engine based on Item Based Collaborative Filtering (IBCF) which searches for the most similar books based on the user ratings. We can download the data from [here](https://drive.google.com/file/d/1WvTmAfO09TCX7xp7uu06__ziic7JnrL5/view?usp=sharing).

In [2]:
book_ratings = pd.read_csv('BX-Book-Ratings.csv',sep=";", encoding="latin")
books = pd.read_csv('BX-Books.csv',sep=";", encoding="latin", error_bad_lines=False)

D:\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:3441: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)
b'Skipping line 6452: expected 8 fields, saw 9\nSkipping line 43667: expected 8 fields, saw 10\nSkipping line 51751: expected 8 fields, saw 9\n'
b'Skipping line 92038: expected 8 fields, saw 9\nSkipping line 104319: expected 8 fields, saw 9\nSkipping line 121768: expected 8 fields, saw 9\n'
b'Skipping line 144058: expected 8 fields, saw 9\nSkipping line 150789: expected 8 fields, saw 9\nSkipping line 157128: expected 8 fields, saw 9\nSkipping line 180189: expected 8 fields, saw 9\nSkipping line 185738: expected 8 fields, saw 9\n'
b'Skipping line 209388: expected 8 fields, saw 9\nSkipping line 220626: expected 8 fields, saw 9\nSkipping line 227933: expected 8 fields, saw 11\nSkipping line 228957: expected 8 fields, saw 10\nSkipping line 245933: expected 8 fields,

* Explore both datasets

In [3]:
books.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


In [4]:
books['ISBN'] = books['ISBN'].astype('str')

In [5]:
book_ratings['ISBN'] = book_ratings['ISBN'].astype('str')

In [6]:
test = books[books['Book-Title'] == 'The Mummies of Urumchi']['ISBN'].values[0]

In [7]:
test

'0393045218'

In [8]:
book_ratings.head()

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


* create dataframe with name 'df_book_features' from book_ratings that have `ISBN` as index, `User-ID` as columns and values are `Book-Rating`.
    - The data are quite big so it's OK to use a sample only in case your PC has limited RAM.


In [13]:
df_book_features = book_ratings.sample(60000).pivot_table(index='ISBN',columns='User-ID',values='Book-Rating').fillna(0)

In [77]:
df_book_features

User-ID,8,22,53,56,75,92,135,176,182,183,...,278633,278637,278668,278771,278773,278782,278795,278843,278851,278854
ISBN,,,,,,,,,,,,,,,,,,,,,
0 907 062 008,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0/330/25864/8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0/385/42473/6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
00000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0000000033220,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
T110105305048,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
X439361760,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"\8887517452\""""",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


* create the instance of the NearestNeighbors class

In [27]:
nn = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=5, n_jobs=-1)

In [28]:
# from scipy.sparse import csr_matrix
# # convert dataframe of book features to scipy sparse matrix
# mat_book_features = csr_matrix(df_book_features.values)

* fit the NearestNeighbors using'df_book_features'

In [29]:
nn.fit(df_book_features)

NearestNeighbors(algorithm='brute', metric='cosine', n_jobs=-1)

In [23]:
import numpy as np

In [90]:
ISBN = '0393045218'

In [84]:
recs = nn.kneighbors(np.array(df_book_features.loc[[ISBN]]).reshape(1, -1), n_neighbors=5, return_distance=False)

In [85]:
recommends = []
for i in recs:
    recommends.append(df_book_features.iloc[i].index)
    

In [86]:
for i in range(0,len(recommends[0])):
    print(recommends[0][i])

0002570807
0765300346
0765300702
076530046X
0765300796


* create function that returns top 5 most similar movies (according to KNN model) for selected ISBN
    * the input will be Book-Title from the DataFrame books 
    * the output will be the Book-Titles of the top 5 most similar books.
    * for every book in the top 5 most similar books, print also the distance from the selected book (ISBN we chose as input to the function)

In [105]:
def make_recommendations(book_title):
    """
    make top 5 book recommendations
    Parameters
    ----------
    fav_book: str, name of user input book
    """
    # get isbn
    ISBN = books[books['Book-Title'] == book_title]['ISBN'].values[0]
    # get recommendations
    recs = nn.kneighbors(np.array(df_book_features.loc[[ISBN]]).reshape(1, -1), n_neighbors=5, return_distance=False)
    # print results
    recommends = []
    for i in recs:
        recommends.append(df_book_features.iloc[i].index)
    rec_isbn = []
    for i in range(0,len(recommends[0])):
        rec_isbn.append(recommends[0][i])
    for i in range(0,len(rec_isbn)):
        print(books[books['ISBN'] == rec_isbn[i]]['Book-Title'].values[0])
        
    return

* Apply the function to book of your choice

In [106]:
make_recommendations('Clara Callan')

War for the Oaks
Wizard's First Rule (Sword of Truth, Book 1)
Conan The Liberator (Conan)
Not Really the Prisoner of Zenda: A Guardians of the Flame Novel
Fitcher's Brides (Fairy Tales)
